In [12]:
import numpy as np
from core import DynamicalSystem
from plotting import evolution_plot, phase_portrait, evolution_dot_plot
from rich.console import Console
from rich.traceback import install
from bokeh.plotting import figure, show, output_notebook 
from bokeh.models import HoverTool, ColumnDataSource
from joblib import Parallel, delayed
from tqdm import tqdm

console = Console()
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x10592ecf0>>

In [13]:
def system(t, state_vector, params):
    x, y, phi = state_vector
    omega, gamma, kappa, eps, Omega, L1, L2, L3 = params
    eta = -kappa/2
    beta = np.tan(gamma)
    nu = omega + eta*beta
    epsc = eps/np.cos(gamma)

    # forcing terms
    Fx = L1*np.cos(Omega*t) + L2*np.cos(3*Omega*t) + L3*np.cos(5*Omega*t)
    Fy = L1*np.sin(Omega*t) + L2*np.sin(3*Omega*t) + L3*np.sin(5*Omega*t)

    f1 = L1*np.sin(1*Omega*t-phi-gamma)
    f2 = L2*np.sin(3*Omega*t-phi-gamma)
    f3 = L3*np.sin(5*Omega*t-phi-gamma)

    # cartesian equations
    xdot = eta*x - nu*y - eta*(x - beta*y)*(x**2 + y**2) + eps*Fx
    ydot = eta*y + nu*x - eta*(y + beta*x)*(x**2 + y**2) + eps*Fy
    phi_dot = omega + epsc*(f1+f2+f3)
    return np.array([xdot, ydot, phi_dot], np.float64)

In [14]:
def frequency(t, state_vector, state_vector_dot):
    x, y, _ = state_vector
    xdot, ydot, _ = state_vector_dot
    freq = (ydot*x - xdot*y)/(x**2 + y**2)
    return freq

In [15]:
parameters = {
    'omega':    1.0,
    'gamma':    0.1,
    'kappa':    -2.0,
    'eps':      0.3,
    'Omega':    0.4,
    'L1':       1.00,
    'L2':       0.10,
    'L3':       0.01
}

# Set initial state and time span
u0 = {'x': 1.0, 'y': 0.0, 'phi': 0.0}

In [16]:
phs = DynamicalSystem(system, t0=0, x0=u0, 
                      parameters=parameters)

In [17]:
phs.integrate(50_000,100)

In [18]:
p = figure(title="Time evolution", 
               x_axis_label="time",
               y_axis_label="variables",
               width=1200,
               height=600)
output_notebook()
p.line(phs.t_sol[::1], phs.xdot_sol[2][::1],
            legend_label="phi",
            line_width=2, color='#344966')
p.hspan(y=[phs.xdot_sol[2].mean()], line_color='#FF6666', line_width=2)
    
p.add_tools(HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")]))
p.legend.location = 'top_left'
show(p)

Loading BokehJS ...

In [20]:
Omega_range = np.linspace(0.2, 0.6, 500)
freq_xy = np.zeros_like(Omega_range)
freq_f1 = np.zeros_like(Omega_range)
run_time = 50_000
tr_time = 100

for i, w in tqdm(np.ndenumerate(Omega_range)):
    phs.reset()
    phs.set_parameter('Omega', w)

    freq_xy[i] = phs.evaluate(frequency, run_time, tr_time).mean()
    freq_f1[i] = phs.xdot_sol[2].mean()

500it [46:45,  5.61s/it]


In [55]:
from bokeh.models import Span
p = figure(title="Synchronization domain", 
               x_axis_label="Omega",
               y_axis_label="freq/Omega",
               width=1200,
               height=600)
output_notebook()
p.line(Omega_range, freq_xy/Omega_range,
            legend_label="Full XY",
            line_width=2, color='#344966')
p.line(Omega_range, freq_f1/Omega_range,
            legend_label="First-order",
            line_width=2, color='#FF6666')

w, eps = parameters['omega'], parameters['eps']
L, gamma = parameters['L2'], parameters['gamma']

shift = 0.00
Omega_min = (w - eps*L/np.cos(gamma) - shift)/3
Omega_max = (w + eps*L/np.cos(gamma) - shift)/3

vline1 = Span(location=Omega_min, dimension='height', 
              line_color='red', line_width=2)
vline2 = Span(location=Omega_max, dimension='height', 
              line_color='red', line_width=2)
vline3 = Span(location=(w - shift)/3, dimension='height', 
              line_color='black', line_width=2)
p.add_layout(vline1)    
p.add_layout(vline2)    
p.add_layout(vline3)    

p.add_tools(HoverTool(tooltips=[("Omega", "@x"), ("Ratio", "@y")]))
p.legend.location = 'top_right'
show(p)

Loading BokehJS ...

In [56]:
phs.reset()
phs.set_parameter('Omega', 0.33)

freq_xy_test = phs.evaluate(frequency, run_time, tr_time).mean()
freq_f1_test = phs.xdot_sol[2].mean()

In [76]:
p = figure(title="Time evolution", 
               x_axis_label="time",
               y_axis_label="variables",
               width=1200,
               height=600)
output_notebook()
p.line(phs.t_sol[::1], 3*0.31241*phs.t_sol[::1]-phs.x_sol[2][::1],
            legend_label="phi",
            line_width=2, color='#344966')
p.hspan(y=[phs.xdot_sol[2].mean()], line_color='#FF6666', line_width=2)
    
p.add_tools(HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")]))
p.legend.location = 'top_left'
show(p)

Loading BokehJS ...